```python
#!/usr/bin/env python
# coding: utf-8 

#   This software component is licensed by ST under BSD 3-Clause license,
#   the "License"; You may not use this file except in compliance with the
#   License. You may obtain a copy of the License at:
#                        https://opensource.org/licenses/BSD-3-Clause
  

'''
Training script of human activity recognition system (HAR), based on Support Vector Machine Classifier or (SVC).
'''
```

# Step by Step HAR using Classical Machine Learning and STM32CubeAI

This notebook provides a step by step demonstration of a simple <u>H</u>uman <u>A</u>ctivity <u>R</u>ecognition (HAR) system, based on a <u>S</u>upport <u>V</u>ector Machine <u>C</u>lassifier (SVC). This script provides a simple data preperation script through `DataHelper` class and let user to preprocess, split, and segment the dataset to bring it into the form which can be used for training and validation of the Machine Learning model. 

All the implementations are done in Python and the machine learning part using [scikit-learn](https://scikit-learn.org/stable/getting_started.html) library.

For demonstration purposes this script uses two datasets created for HAR using accelerometer sensor. 

* A public dataset provided by <u>WI</u>reless <u>S</u>ensing <u>D</u>ata <u>M</u>ining group named as **<u>WISDM</u>**. The details of the dataset are available [here](http://www.cis.fordham.edu/wisdm/dataset.php).

* Our own propritery dataset called **<u>AST</u>**. 

**Note**: We are not providing any dataset in the function pack. The user can download WISDM dataset from [here](http://www.cis.fordham.edu/wisdm/dataset.php), while **<u>AST</u>** is a private dataset and is not provided. Although a subset of the **<u>AST</u>** dataset is provided in the function pack at location `/FP-AI-MONITOR1/Utilities/AI_Resources/Datasets/AST/`.

Following figure shows the detailed workflow of HAR based on classical machine learning.


<p align="center">
<img width="760" height="400" src="workflow_ml.png">    
</p>

Now, let us implement it step by step.

### Step1 : Import necessary dependencies
Following section imports all the required dependencies. This also sets seeds for random number generator in Numpy to make the results more deterministic between different runs.

In [ ]:
# python libraries
import os, matplotlib.pyplot as plt, numpy as np, warnings
from datetime import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable

# scikit-learn modules
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix

# private libraries
from PrepareDataset import DataHelper
import ML_Utilities
from SVC_Utilities import *

# disabling annoying warnings originating from python
warnings.simplefilter("ignore")
# setting the seeds to the random generators of Numpy and Tensorflow
np.random.seed( 611 )

### Step2: Set environment variables
Following section sets some user variables which will later be used for:
* preparing the dataset for training and testing the machine learning model, and
* post conversion validation of the machine learning model

In [ ]:
# data variables
dataset = 'AST'
# dataset = 'WISDM'
reducedClasses = True
segmentLength = 24
stepSize = 24
preprocessing = True

# training variables
trainTestSplit = 0.6
trainValidationSplit = 0.7
nrSamplesPostValid = 2

### Step3: Result directory
Each run can have different variables depending on user settings and to compare the results of different choices, such as different segment size for the window for data, different overlap settings etc. We can save the results to compare different runs. 

Following section creates a result directory to save results for the current run. The name of the directory has following format. `Mmm_dd_yyyy_hh_mm_ss`, and example name for directory can be `Jul_20_2021_14_31_20/`. In every result directory there will a `.txt` file with information and an `onnx` model called `SVC_model.onnx`.

In [ ]:
# if not already exist create a parent directory for results.
if not os.path.exists( './results_svm/'):
    os.mkdir( './results_svm/' )
svmResDir = 'results_svm/{}/'.format(datetime.now().strftime( "%Y_%b_%d_%H_%M_%S" ) )
if not os.path.exists( svmResDir):
    os.mkdir( svmResDir )
infoString = 'runTime : {}\nDatabase : {}\nModel : {}\nSeqLength : {}\nStepSize : {}\nReducedClasses : {}'.\
format( datetime.now().strftime("%Y-%b-%d at %H:%M:%S"), dataset, 'SVC', segmentLength, stepSize, reducedClasses )
with open( svmResDir + 'info.txt', 'w' ) as text_file:
    text_file.write( infoString )

### Step4: Create a `DataHelper` object
The script in the following section creates a `DataHelper` object to preprocess, segment and split the dataset as well as to create the integer labels for the outputs to make the data training and testing ready using the choices set by the user in **Step2**.

For example in case of **AST** dataset 
* the label `Stationary` will change to `0`.
* the label `Walking` will change to `1`.
* the label `Jogging` will change to `2`.
* the label `Biking` will change to `3`.


In [ ]:
myDataHelper = DataHelper( dataset = dataset, reducedClasses = reducedClasses, 
                          seqLength = segmentLength, seqStep = stepSize, preprocessing = preprocessing,
                          trainTestSplit = trainTestSplit, trainValidSplit = trainValidationSplit, resultDir = svmResDir )

### Step5: Prepare the dataset
Following section prepares the dataset and create four tensors namely `trainX`, `trainy`, `testX`, `testy`. Each of the variables with trailing `X` are the inputs with shape `[_, ( segmentLength  * 3 ), 1 ]`and each of the variables with trailing `y` are corresponding outputs with shape `[ , _ ]`, where `_` correspond to the number of samples.

In [ ]:
trainX, trainy, testX, testy = myDataHelper.prepare_dataset_for_SVM()

#### Step5.1: print the number of samples in train, test and validation data sets and the number of classes

In [ ]:
print( 'Number of training samples : {}\nNumber of test samples : {}\nNumber of classes : {}'.\
      format( trainX.shape[ 0 ], testX.shape[ 0 ], len( myDataHelper.classes ) ) )

### Step6: SVC
```python
# an example call to initialize an SVC object
sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=None
```

#### Step6.1: Run the grid search to find the optimized hyper parameters for SVC
SVM based classifier object can be created with a set of parameters. Although a set of default parameter are provided the performance strongly depend on these parameters. The following code uses a function in `SVCUtilities.py` script to find the best parameters out of the grid. These parameters are choosen to provide the best peformance. Following section computes and prints these values.

**Note**: The search of these parameters can take long time, we have provided precomputed values for the sake of running scripts quickly. Setting `precomputed = True` value will result in return of these precomputed values. Setting this to `False` will result in a computation of these parameters.

In [ ]:
bestParams = ParamSearch( trainX, trainy, precomputed = True ) # set precomputed to false if you want to perform grid search
print( bestParams )

#### Step6.2: Creating an SVC model with just 2% of the training data and seeing the performance

##### Step6.2.1: initializing the SVC

The following script initializes the an object of SVC using the best parameters computed from the section above.

In [ ]:
svc_model = SVC( kernel = bestParams[ 'kernel' ], C = bestParams[ 'C' ], gamma = bestParams[ 'gamma' ], 
          decision_function_shape='ovo', probability = True ) 

##### Step6.2.2: fitting the svc_model to the 2% of the data
Unlike a training in Neural networks which required multiple iterations over the data, when performing a training of the traditional machine learning model a fit function is enough to call.

One of the power of the traditional machine learning algorithm is that they can be very performant when a small dataset is available to train. To demonstrate that, the following section uses only the `2%` of the training dataset for the training purpose.

In [ ]:
subsetInd = max( int( len( trainX ) / 50 ), 100 ) # if number of samples are lesser than 5000 use 100 samples
svc_model.fit( trainX[ : subsetInd ], trainy[ : subsetInd ] );

##### Step6.2.3: predicting the accuracy of the svc model
Once the model is fit the next step is to test it and see the accuracy of the model on the `full` train dataset and the test dataset.

In [ ]:
print( 'train accuracy', np.round( 100 * accuracy_score( trainy, svc_model.predict( trainX ) ), 2 ) )
print( 'test accuracy', np.round( 100 * accuracy_score( testy, svc_model.predict( testX ) ), 2 ) )

##### Step6.2.4: Saving the model
The model can be saved as onnx model now using the `ML_Utilities.py` script. More specifically `model_conversion_to_onnx`.

In [ ]:
ML_Utilities.model_conversion_to_onnx( svc_model, trainX, svmResDir + 'svc_2_pct.onnx', svmResDir + 'svc_2_pct.json' )

### Step7: Dimensionality reduction of the data before machine learning model
Instead of using the raw data itself, we can be smarter and use the dimensionality reduction based on PCA or SVD.
This dimensionality reduction has two fold benefits, which are:
* improved accuracy by removing the unrelevant information,
* reduced sizes of the model by having fewer data to save.

#### Step7.1: building a pipeline for the machine learning model along with the dimensional reduction
Thanks to `scikit-learn` pipeline functionality it is quite easy to implement multiple signal processing functions in a single object. Following we are building a pipeline object which implements `Dimensionality_reduction` and `SVC` at once like a pipeline.

In [ ]:
# we are going to use 24 samples instead of all 72
dim_reduction = TruncatedSVD( n_components = 24, algorithm = 'arpack', random_state = None )
svc_model = Pipeline( [ ( 'dim_reduction', dim_reduction ),
                       ('svc', SVC( kernel = bestParams['kernel'], C = bestParams['C'], gamma = bestParams['gamma'], 
          decision_function_shape='ovo', probability = True ) ) ] )    

#### Step7.1.2: Training the pipeline model
To show the benefits of the dimensionality reduction we are going to traing the new model with the very same data and see the increase in the performance and reduction in the model size.

Following section trains the model with dimensionality reduction and with just `2%` of the training data.

In [ ]:
svc_model.fit( trainX[ : subsetInd ], trainy[ : subsetInd ] );

#### Step7.1.3: Testing the pipeline model
Following section tests the accuracy of the trained pipeline model on the `full` training and test datasets.

In [ ]:
print( 'train accuracy', np.round( 100 * accuracy_score( trainy, svc_model.predict( trainX ) ), 2 ) )
print( 'test accuracy', np.round( 100 * accuracy_score( testy, svc_model.predict( testX ) ), 2 ) )

#### Step7.1.4: Testing the pipeline model
Following section saves the trained model. One can compare the performance and the size of the models to see the benefits of the dimensionality reduction before applying machine learning model.

In [ ]:
ML_Utilities.model_conversion_to_onnx( svc_model, trainX, svmResDir + 'svc_2_pct_dim_red.onnx', svmResDir + 'svc_2_pct_dim_red.json' )

### Step8: Validating the created model on the test data
Following we plot the confusion matrix of the prediction of the model on the test data to understand the distribution of the errors.

In [ ]:
cM = confusion_matrix( testy, svc_model.predict( testX ) )
cmNormalized = np.round( 100 * cM / np.sum( cM, axis = 1 ), 2 )

#### creating the confusion matrix figure

In [ ]:
title = 'Confusion matrix \n Nr. of Occurences and \n %age confidence'
# creating a figure object
fig, ax = plt.subplots( figsize = ( 7, 7 ) )
# plotting the confusion matrix
im = ax.imshow( cmNormalized, interpolation = 'nearest', cmap = plt.cm.Blues )

# assiging the title, x and y labels
plt.xlabel( 'Predicted Values' )
plt.ylabel( 'Ground Truth' )
plt.title( title )

# defining the ticks for the x and y axis
plt.xticks( range( len( myDataHelper.classes ) ), myDataHelper.classes, rotation = 60 )
plt.yticks( range( len( myDataHelper.classes ) ), myDataHelper.classes )
plt.ylim( bottom = len( myDataHelper.classes ) - .5 )
plt.ylim( top = -0.5 )

# annotating the confusion matrix with values and printing in shall the accuracy for each class
width, height = cM.shape 

print( 'Accuracy for each class is given below.' )
for predicted in range( width ):
    for real in range( height ):
        if cmNormalized[ predicted,real ] > 45:
            color = 'white'
        else:
            # background will be two light to have a white colored text
            color = 'black'
        
        if( predicted == real ):
            print( myDataHelper.classes[ predicted ].ljust( 12 )+ ':', cmNormalized[ predicted,real ], '%' )
        plt.gca( ).annotate( '{}\n{}%'.format( cM[ predicted, real ], cmNormalized[ predicted, real ] ), xy = ( real, predicted ),
            horizontalalignment = 'center', verticalalignment = 'center', color = color )

# showing colorbar for normalized confusion matrix case
# creating a color bar and setting the limits
divider = make_axes_locatable( plt.gca( ) )
cax = divider.append_axes( "right", "5%", pad="3%" )
plt.colorbar( im, cax = cax )

# making sure that the figure is not clipped
plt.tight_layout( )

# showing plot
plt.savefig( '{}svc_2_pct_dim_red_confusion_matrix.png'.format( svmResDir )  )
plt.show( )

### Step9: Validating <u>SVC</u> model created on dataset using data logged using High Speed Datalogger on STWIN
Following script generates segments from a High Speed Datalogger binary provided in `/FP-AI-MONITOR1/Utilities/Datalog/` and evaluates the performance of the trained model on it.

#### Stationary activitiy

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_ml_segments( '../../Datasets/HSD_Logged_Data/HAR/Stationary/', 
                                                         activityName = 'Stationary' )


print( 'Validation accuracy', 
      round( 100 * accuracy_score( labels, svc_model.predict( segments ) ), 2 ), '%' )

#### Walking activity

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_ml_segments( '../../Datasets/HSD_Logged_Data/HAR/Walking/', 
                                                         activityName = 'Walking' )


print( 'Validation accuracy', 
      round( 100 * accuracy_score( labels, svc_model.predict( segments ) ), 2 ), '%' )

#### Jogging activity

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_ml_segments( '../../Datasets/HSD_Logged_Data/HAR/Jogging/', 
                                                         activityName = 'Jogging' )


print( 'Validation accuracy', 
      round( 100 * accuracy_score( labels, svc_model.predict( segments ) ), 2 ), '%' )

#### Biking activity

This section will only run when AST dataset is used which have `Biking` class. if WISDM dataset is used this section will generate an error as the `Biking` does not exist.

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_ml_segments( '../../Datasets/HSD_Logged_Data/HAR/Biking/', 
                                                         activityName = 'Biking' )


print( 'Validation accuracy', 
      round( 100 * accuracy_score( labels, svc_model.predict( segments ) ), 2 ), '%' )

### Step10: Create an npz file for validation after conversion from CubeAI.

In [ ]:
myDataHelper.dump_data_for_post_validation( testX, testy, nrSamplesPostValid )

### Additional work to study the size of the dataset to obtain 85% performance
This section is written to compare the performance of the **<u>CNN</u>** vs **<u>SVC</u>**.
As the idea is that when small dataset is available we do not need the CNN for good performance instead SVC can be an easy approach to have comparable or even better performances.

##### Note: This section only works with the full dataset, i.e. when there are enough samples available

In [ ]:
if not os.path.isdir( svmResDir ):
    os.mkdir(svmResDir)
print('samples,train_acc,test_acc')
dim_reduction = TruncatedSVD( n_components = 24, algorithm = 'arpack', random_state = None )
model = Pipeline( [ ( 'dim_reduction', dim_reduction ),
                   ( 'svc', SVC( kernel = bestParams['kernel'], C = bestParams['C'], gamma = bestParams['gamma'],
                    decision_function_shape='ovo', probability = True ) ) ] )
if( trainX.shape[0] > 4000 ):
    for ii in range( 100, 4000, 100 ):
        model.fit( trainX[:ii], trainy[:ii] )
        train_acc =  np.round( 100 * accuracy_score( trainy, model.predict( trainX ) ), 2 )
        test_acc =  np.round( 100 * accuracy_score( testy, model.predict( testX ) ), 2 )
        print( '{},{},{}'.format( ii, train_acc, test_acc ) )
else:
    print('Not enough samples available for this study')


```python
### code to generate the graph below
import pandas as pd
data = pd.read_csv('./svm_vs_nn.txt', delimiter = ',', dtype = 'a' )
fig = plt.figure(figsize = (20,10))
plt.plot( data['samples'].values.astype(np.float32), data['svm_tt'].values.astype(np.float32), linewidth = 5 )
plt.plot( data['samples'].values.astype(np.float32), data['nn_tt'].values.astype(np.float32), linewidth = 5 )
plt.legend([ 'SVC', 'CNN' ], fontsize = 25)
plt.xlabel('Training Samples', fontsize = 25)
plt.ylabel('Accuracy [%]', fontsize = 25)
plt.title('SVC vs CNN', fontsize = 35)
plt.xticks(range(500, 4000, 500 ), fontsize = 25)
plt.yticks(range(50, 110, 10 ), fontsize = 25)
plt.grid()
plt.show()
plt.savefig('svc_vs_cnn_acc.jpg')
```
<p align="center">
<img width="900" height="400" src="svc_vs_cnn_acc.jpg">    
</p>
<b>Note : </b> These graphs are generated using the full AST dataset.